In [9]:
import numpy as np
import cv2

In [10]:
s2 = cv2.imread("Images/2_colour.jpeg") #Scene with 2 objects
mask = cv2.imread("Images/masks/souvenirs_no_3_colour_mask_2_mask.png") #Target object mask

In [11]:
def ExtractObject(S2, ObjectMask):
    """Extracts an object from an image scene based on the mask used"""

    final_im = ObjectMask*S2
    final_im = cv2.bitwise_not(final_im)
    return final_im

In [ ]:
def ApplyFilter(ExtractedObject, FilterIndex):
    """Applies convolution on the object using pre-defined image kernels"""

    if FilterIndex==0: #Apply no filter
        FilteredExObject = ExtractedObject
    elif FilterIndex==1:
        #Will need to define some kernels and use them here
    elif FilterIndex==2:
        #Will need to define some kernels and use them here
    return FilteredExObject

In [33]:
def ObjectBlender(S1, FilteredExObject):
    """Adds the filtered extracted object to the image in scene S1"""

    alpha = 0.5
    beta = 1.0 - alpha
    BlendingResult = cv2.addWeighted(S1, alpha, FilteredExObject, beta, 0.0)
    cv2.imshow('Blended', BlendingResult)
    cv2.waitKey(0)
    return BlendingResult

In [ ]:
def CompareResult(BlendingResult, S2, metric):
    """Compares the blended image with the scene with 2 objects"""

    if metric==1: #Sum of Squared Distance Error (SSD)
        error = BlendingResult-S2
        error = sum(error(:).^2)
    elif metric==2: #Mean Squared Error (MSE)
        error = np.sum((BlendingResult.astype("float") - S2.astype("float")) ** 2)
        error /= float(BlendingResult.shape[0] * BlendingResult.shape[1])
    return error

In [34]:
extracted = ExtractObject(image, mask)

In [35]:
s1 = cv2.imread("Images/1_colour.jpeg", 1) #Scene with 1 object
blended = ObjectBlender(s1, extracted)